# ROC and AUC 

This notebook covers the basics of Receiver Operator Characteristic (ROC) graphs and the concept of the area under the curve (AUC).

Source: "ROC and AUC, Clearly Explained!", StatQuest with Josh Starmer https://www.youtube.com/watch?v=4jRBRDbJemM

Data Science Project Steps:

    1) Load the data

    2) Define appropriate X and y

    3) Normalize the Data

    4) Split the data into train and test sets.

    5) Fit a model

    6) Predict

## Part 1: Model Confusion Matrix

![Binary Classifiction Confusion Matrix](supporting_images/Binary_Classifiction_Confusion_Matrix.jpg)


## Part 2: Specificity and Sensitivity

Sensitivity is the model's ability to correctly identify positives (belongs to a class). Specificity is the model's ability to correctly identify negatives (does not belong to a class)

